In [ ]:
from scipy.stats import norm

import math
import numpy as np
import pandas as pd

In [ ]:
def probab(S0: float,
           mu: float,
           vol: float,
           T: float,
           lower: float = None,
           upper: float = None):

    # 1. GBM의 로그 가격 분포 파라미터 계산
    # ln(S_T) ~ N(mean, std^2)
    mean = math.log(S0) + (mu - vol ** 2 / 2) * T
    std = vol * math.sqrt(T)

    # 2. 하한값(lower) 처리
    if lower is None or (isinstance(lower, float) and np.isnan(lower)) or lower <= 0:
        log_lower = -math.inf
    else:
        log_lower = math.log(lower)
        
    # 3. 상한값(upper) 처리
    if upper is None or (isinstance(upper, float) and np.isnan(upper)):
        log_upper = math.inf
    else:
        log_upper = math.log(upper)

    # 4. 표준정규분포를 이용한 확률 계산
    return norm.cdf((log_upper - mean) / std) - norm.cdf((log_lower - mean) / std)
    

In [ ]:
array = np.arange(0.7, 0.91, 0.05)

In [ ]:
array

In [ ]:
array = np.pad(array, (1, 1), constant_values=None)
array

In [ ]:
result = np.column_stack((array[:-1], array[1:]))
result

In [ ]:
np.nan is None

In [ ]:
for lower, upper in result:
    print(f"lower: {lower}, upper: {upper}")
    print(probab(S0=0.8, mu=0.0, vol=0.12, T=0.25, lower = lower, upper=upper))

In [ ]:
0.014063043907057513 + 0.13380012690632984 + 0.36410330260072526 + 0.33895888215036674 + 0.1259466908321788 + 0.023127953603341855

In [ ]:
# TODO gemini 코드 개선해보기

import math
import numpy as np
from scipy.stats import norm

def probab(S0: float, mu: float, vol: float, T: float,
           lower: float = None, upper: float = None):
    
    # 1. GBM의 로그 가격 분포 파라미터 계산
    # ln(S_T) ~ N(mean, std^2)
    mean = math.log(S0) + (mu - vol ** 2 / 2) * T
    std = vol * math.sqrt(T)

    # 2. 하한값(lower) 처리
    if lower is None or (isinstance(lower, float) and np.isnan(lower)) or lower <= 0:
        log_lower = -math.inf
    else:
        log_lower = math.log(lower)

    # 3. 상한값(upper) 처리
    if upper is None or (isinstance(upper, float) and np.isnan(upper)):
        log_upper = math.inf
    else:
        log_upper = math.log(upper)

    # 4. 표준정규분포를 이용한 확률 계산
    return norm.cdf((log_upper - mean) / std) - norm.cdf((log_lower - mean) / std)

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import lognorm

def plot_stock_distribution(S0, mu, vol, T, price_points):
    # 1. 로그정규분포 파라미터 계산
    # s: 로그정규분포의 형상 매개변수 (표준편차)
    # scale: 로그정규분포의 척도 매개변수 (중앙값, exp(mean))
    sigma_t = vol * np.sqrt(T)
    mu_t = np.log(S0) + (mu - 0.5 * vol**2) * T
    
    # 2. X축 범위 설정 (주가 0부터 마지막 구간의 2배까지)
    max_p = max(price_points) * 1.5
    x = np.linspace(0.1, max_p, 500)
    
    # 3. PDF(확률 밀도 함수) 계산
    # scipy의 lognorm은 s=sigma, scale=exp(mu)로 정의됨
    pdf = lognorm.pdf(x, s=sigma_t, scale=np.exp(mu_t))
    
    # 4. 그래프 그리기 시작
    plt.figure(figsize=(12, 6))
    plt.plot(x, pdf, 'b-', lw=2, label=f'Stock Price PDF at T={T}')
    
    # 5. 우리가 설정한 구간(price_points)에 수직선 긋기
    colors = plt.cm.viridis(np.linspace(0, 1, len(price_points)))
    for p, c in zip(price_points, colors):
        plt.axvline(x=p, color=c, linestyle='--', alpha=0.7, label=f'Price: {p}')
        
    # 6. 구간별 색 채우기 (확률 면적 시각화)
    # 예: 첫 번째 구간 (0 ~ 10)
    plt.fill_between(x, pdf, where=(x <= price_points[0]), color='gray', alpha=0.2)
    # 중간 구간들
    for i in range(len(price_points)-1):
        plt.fill_between(x, pdf, where=(x >= price_points[i]) & (x <= price_points[i+1]), 
                         alpha=0.3, label=f'Interval {i+1}')
    
    # 7. 그래프 꾸미기
    plt.title(f'Stock Price Distribution ($S_T$) and Probability Intervals', fontsize=14)
    plt.xlabel('Stock Price ($S$)', fontsize=12)
    plt.ylabel('Density', fontsize=12)
    plt.legend(loc='upper right', bbox_to_anchor=(1.2, 1))
    plt.grid(axis='y', alpha=0.3)
    plt.show()

# --- 실행 ---
S0, mu, vol, T = 30, 0.05, 0.2, 1.0
price_points = [10, 20, 30, 40, 50]

plot_stock_distribution(S0, mu, vol, T, price_points)